In [ ]:
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt

In [ ]:
# Define the map size and range of the landmark coordinates
map_size = 100
landmark_range = (-map_size/2, map_size/2)

# Generate 1000 landmark coordinates randomly
num_landmarks = 100
landmark_coords = np.random.uniform(low=landmark_range[0], high=landmark_range[1], size=(2, num_landmarks))

# Add the landmark coordinates to the state vector
state_vector_size = 3 
state_vector = np.zeros((state_vector_size, 1))

for k in range(len(landmark_coords.T)):
    state_vector=np.append(state_vector,landmark_coords[:,k].reshape(-1,1),axis=0)

In [ ]:
# Covariance matrix (initialized with large values)
P = np.eye(state_vector_size) * 1e6
P[0][0],P[1][1],P[2][2]=0,0,0

#process and measurement noise covariance matrices
Rt = 5*np.array([[0.01,0,0],
               [0,0.01,0],
               [0,0,0.01]])

Qt = np.array([[0.01,0],
               [0,0.01]])

Fx=np.concatenate((np.identity(3),np.zeros((3,2*num_landmarks))),axis=1)
FxT=np.transpose(Fx)


In this example, we assume that the measurements of the robot's position and the landmark position are subject to Gaussian noise with a standard deviation of 0.1 meters in both the x and y directions. The resulting measurement noise covariance matrix R is a 2x2 matrix that captures the variance of the noise in each dimension of the measurement vector.

Note that these examples are just one possible choice of process noise covariance and measurement noise covariance matrices for a simple EKF SLAM scenario. The actual choice of Q and R will depend on the specifics of the problem being solved and the characteristics of the noise in the system.

In [ ]:
def motion_model(state, control, dt):
   


  #u = [v, w] => velocity, angular velocity
  v=control[0]
  w=control[1]

  I=np.identity(3+2*num_landmarks)
       
  xx =  - (v/w)*math.cos(state[2])+(v/w)*math.cos(state[2]+w*dt)
  yy = -(v/w)*math.cos(state[2])+(v/w)*math.cos(state[2]+w*dt)
  thetaa = 0

  arr=np.array([[0,0,xx],[0,0,yy],[0,0,thetaa]]) 
                
  G=I+FxT@arr@Fx
       
  x =  - (v/w)*math.sin(state[2])+(v/w)*math.sin(state[2]+w*dt)+Rt[0][0] #Adding noise
  y = (v/w)*math.cos(state[2])-(v/w)*math.cos(state[2]+w*dt)+Rt[1][1]
  theta =  w*dt+Rt[2][2]  
                
        
  state=state+ np.matmul(FxT,np.array([[x], [y], [theta]]))
        
  return state,G

In [ ]:
def obs_model(state, landmark_id, dt):
              
  # Predict the expected measurement of a landmark based on the robot's pose
  x, y, theta = state[0], state[1], state[2]
  lx, ly = state[3 + 2 * landmark_id], state[3 + 2 * landmark_id + 1]

  delta=np.array([[lx-x],[ly-y]])
  q=delta.T@delta

  range = np.sqrt(q)
  bearing = np.arctan2(delta[1], delta[0]) - theta

  H=(1/q)*np.array([[-1*np.sqrt(q)*delta[0],-1*np.sqrt(q)*delta[1],0,np.sqrt(q)*delta[0],np.sqrt(q)*delta[1]],
                    [delta[1],-delta[0],-q,-delta[1],delta[0]]])
  
  Fj=np.zeros((5,2*num_landmarks+3))
  Fj[:3,:3]=np.eye(3)
  Fj[3][3+2*landmark_id],Fj[4][4+2*landmark_id]=1,1

  H=H@Fj

  range += Qt[0][0]
  bearing+=Qt[1][1]

  return np.array([range, bearing]).reshape((2, 1)),H

In [ ]:
# initialize landmarks_seen list
landmarks_seen = [False] * num_landmarks

In [ ]:
def ekf_slam(state,landmark_id, control, dt,P,landmarks_seen):
  state_new,Gt=motion_model(state, control, dt)
  P_new=Gt@P@np.transpose(Gt)+FxT@Rt@Fx

  # check if the landmark has been seen before
  if not landmarks_seen[landmark_id]:
      landmarks_seen[landmark_id] = True
      zt_bar,Ht=obs_model(state_new, landmark_id, dt)

  #Kalman gain
  K=P_new@Ht.T@np.linalg.inv(Ht@P_new@Ht.T+Qt)

  #Update state_vector
  deltaz=np.array([[Qt[0][0]],[Qt[0][0]]])
  state=state_new+K@deltaz

  #Update covariance_metrix
  P=(np.eye(2*num_landmarks+3)-K@Ht)@P_new
  
  return state,P,landmarks_seen

In [ ]:
def plot_map(state, landmark_coords):
    # Plot the landmarks
    plt.scatter(landmark_coords[0, :], landmark_coords[1, :], marker='*', color='red')

    # Plot the robot's current pose
    plt.plot(state[0], state[1], marker='o', color='blue')

    # Set the x and y limits of the plot to show the entire map
    plt.xlim([-map_size/2, map_size/2])
    plt.ylim([-map_size/2, map_size/2])

    # Set the title and labels for the plot
    plt.title('EKF SLAM')
    plt.xlabel('x')
    plt.ylabel('y')

    # Show the plot
    plt.show()

In [ ]:
# Loop through the map and capture landmarks
plot_map(state_vector[:3], landmark_coords)
for i in range(100):
    # Move the robot in a circular path
    v = 1
    w = 0.1
    dt = 0.1
    control = np.array([v, w])
    robot_pose, Gt = motion_model(state_vector, control, dt)


    # Capture nearby landmarks
    for j in range(num_landmarks):
        distance = np.linalg.norm(robot_pose[0:2] - landmark_coords[:, j])
        if distance < 5:
            zt, Ht  = obs_model(robot_pose, j, dt)
            state_vector, P,landmarks_seen = ekf_slam(robot_pose, j, control, dt, P,landmarks_seen)
            plot_map(state_vector[:3], landmark_coords)

In [ ]:
plot_map(state_vector[:3], landmark_coords)
for i in range(100):
    # Move the robot in a circular path
    v = 1
    w = 0.1
    dt = 0.1
    control = np.array([v, w])
    robot_pose, Gt = motion_model(state_vector, control, dt)

    # Update the covariance matrix
    P = Gt @ P @ Gt.T + FxT @ Rt @ Fx

    # Capture nearby landmarks
    for j in range(num_landmarks):
        distance = np.linalg.norm(robot_pose[0:2] - landmark_coords[:, j])
        if distance < 5:
            zt, Ht = obs_model(robot_pose, j, dt)
            state, P = ekf_slam(robot_pose, j, control, dt, P)
            landmark_coords[:, j] = state[3 + 2 * j: 3 + 2 * j + 2].flatten()

    
    state_vector, P = ekf_slam(state_vector, control_vector[t], dt, P)
    plot_map(state_vector[:3], landmark_coords)

    # Update the plot with the robot's new pose
    plot_map(state_vector[:3], landmark_coords)

In [ ]:

# Define the video size and frame rate
video_size = (640, 480)
frame_rate = 10

# Create a video writer object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter('./slam_video.mp4', fourcc, frame_rate, video_size)

# Loop through the map and capture landmarks
for i in range(100):
    # Move the robot in a circular path
    v = 1
    w = 0.1
    dt = 0.1
    control = np.array([v, w])
    robot_pose, Gt = motion_model(state, control, dt)

    # Update the covariance matrix
    P = Gt @ P @ Gt.T + FxT @ Rt @ Fx

    # Capture nearby landmarks
    for j in range(num_landmarks):
        distance = np.linalg.norm(robot_pose[0:2] - landmark_coords[:, j])
        if distance < 5:
            zt, Ht, Fj = obs_model(robot_pose, j, dt)
            state, P = ekf_slam(robot_pose, j, control, dt, P)
            landmark_coords[:, j] = state[3 + 2 * j: 3 + 2 * j + 2].flatten()

    
    # Plot the robot and landmarks on the map
    map_img = np.zeros((map_size, map_size, 3), dtype=np.uint8)
    robot_x, robot_y = int(robot_pose[0, 0] + map_size/2), int(robot_pose[1, 0] + map_size/2)
    map_img[robot_y, robot_x, :] = (255, 255, 255)
    for j in range(num_landmarks):
        landmark_x, landmark_y = int(landmark_coords[0, j] + map_size/2), int(landmark_coords[1, j] + map_size/2)
        map_img[landmark_y, landmark_x, :] = (0, 0, 255)

    # Resize the map image and write to the video file
    resized_map_img = cv2.resize(map_img, video_size)
    video_writer.write(resized_map_img)

# Release the video writer object
video_writer.release()